<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
import datalab.bigquery as bq
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [2]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [3]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [4]:
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [5]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

### Create feature columns for estimator

In [6]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [7]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbf55a983c8>, '_master': '', '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_task_type': 'worker', '_model_dir': 'taxi_trained', '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_task_id': 0, '_train_distribute': None, '_session_config': None, '_keep_checkpoint_max': 5}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 23227.691, st

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [8]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-21-03:41:27
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-21-03:41:27
INFO:tensorflow:Saving dict for global step 608: average_loss = 109.164764, global_step = 608, loss = 12982.81
RMSE on dataset = 10.44819450378418


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [9]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([11.043583], dtype=float32)}
{'predictions': array([11.041], dtype=float32)}
{'predictions': array([11.041875], dtype=float32)}
{'predictions': array([11.039393], dtype=float32)}
{'predictions': array([11.043319], dtype=float32)}
{'predictions': array([11.043096], dtype=float32)}
{'predictions': array([11.0416565], dtype=float32)}
{'predictions': array([11.041688], dtype=float32)}
{'predictions': array([11.043474], dtype=float32)}
{'predictions': array([11.041298], dtype=float32)}
{'predictions': array([11.043584], dtype=float32)}
{'predictions': array([11.043752], dtype=float32)}
{'predictions': array([11.036937], dtype=float32)}
{'predictions': array([11.040911], dtype=float32)}
{'predictions': array([11

{'predictions': array([11.039002], dtype=float32)}
{'predictions': array([11.04499], dtype=float32)}
{'predictions': array([11.037585], dtype=float32)}
{'predictions': array([11.0416765], dtype=float32)}
{'predictions': array([11.043887], dtype=float32)}
{'predictions': array([11.041207], dtype=float32)}
{'predictions': array([11.0451975], dtype=float32)}
{'predictions': array([11.041838], dtype=float32)}
{'predictions': array([11.04224], dtype=float32)}
{'predictions': array([11.100437], dtype=float32)}
{'predictions': array([11.041215], dtype=float32)}
{'predictions': array([11.154532], dtype=float32)}
{'predictions': array([11.207592], dtype=float32)}
{'predictions': array([11.097873], dtype=float32)}
{'predictions': array([11.042866], dtype=float32)}
{'predictions': array([11.042124], dtype=float32)}
{'predictions': array([11.037522], dtype=float32)}
{'predictions': array([11.0405855], dtype=float32)}
{'predictions': array([11.043409], dtype=float32)}
{'predictions': array([11.0410

{'predictions': array([11.098137], dtype=float32)}
{'predictions': array([11.040901], dtype=float32)}
{'predictions': array([11.041326], dtype=float32)}
{'predictions': array([11.040431], dtype=float32)}
{'predictions': array([11.040518], dtype=float32)}
{'predictions': array([11.099214], dtype=float32)}
{'predictions': array([11.041985], dtype=float32)}
{'predictions': array([11.041822], dtype=float32)}
{'predictions': array([11.042849], dtype=float32)}
{'predictions': array([11.040833], dtype=float32)}
{'predictions': array([11.097759], dtype=float32)}
{'predictions': array([11.2684145], dtype=float32)}
{'predictions': array([11.042726], dtype=float32)}
{'predictions': array([11.037578], dtype=float32)}
{'predictions': array([11.209552], dtype=float32)}
{'predictions': array([11.042998], dtype=float32)}
{'predictions': array([11.03782], dtype=float32)}
{'predictions': array([11.2671795], dtype=float32)}
{'predictions': array([11.323387], dtype=float32)}
{'predictions': array([11.0266

{'predictions': array([11.042712], dtype=float32)}
{'predictions': array([11.040723], dtype=float32)}
{'predictions': array([11.04483], dtype=float32)}
{'predictions': array([11.045579], dtype=float32)}
{'predictions': array([11.03663], dtype=float32)}
{'predictions': array([11.041187], dtype=float32)}
{'predictions': array([11.037297], dtype=float32)}
{'predictions': array([11.093543], dtype=float32)}
{'predictions': array([11.043609], dtype=float32)}
{'predictions': array([11.040428], dtype=float32)}
{'predictions': array([11.041509], dtype=float32)}
{'predictions': array([11.099696], dtype=float32)}
{'predictions': array([11.086097], dtype=float32)}
{'predictions': array([11.091806], dtype=float32)}
{'predictions': array([11.042197], dtype=float32)}
{'predictions': array([11.0990305], dtype=float32)}
{'predictions': array([11.042588], dtype=float32)}
{'predictions': array([11.04209], dtype=float32)}
{'predictions': array([11.09743], dtype=float32)}
{'predictions': array([11.043624],

{'predictions': array([11.038233], dtype=float32)}
{'predictions': array([11.042858], dtype=float32)}
{'predictions': array([11.041398], dtype=float32)}
{'predictions': array([11.040448], dtype=float32)}
{'predictions': array([11.267674], dtype=float32)}
{'predictions': array([11.041259], dtype=float32)}
{'predictions': array([11.098153], dtype=float32)}
{'predictions': array([11.041255], dtype=float32)}
{'predictions': array([11.036436], dtype=float32)}
{'predictions': array([11.097463], dtype=float32)}
{'predictions': array([11.100563], dtype=float32)}
{'predictions': array([11.041217], dtype=float32)}
{'predictions': array([11.154546], dtype=float32)}
{'predictions': array([11.098353], dtype=float32)}
{'predictions': array([11.155541], dtype=float32)}
{'predictions': array([11.099065], dtype=float32)}
{'predictions': array([11.097105], dtype=float32)}
{'predictions': array([11.041619], dtype=float32)}
{'predictions': array([11.0412], dtype=float32)}
{'predictions': array([11.041634]

{'predictions': array([11.0434475], dtype=float32)}
{'predictions': array([11.267038], dtype=float32)}
{'predictions': array([11.042525], dtype=float32)}
{'predictions': array([11.038521], dtype=float32)}
{'predictions': array([11.042562], dtype=float32)}
{'predictions': array([11.042751], dtype=float32)}
{'predictions': array([11.212244], dtype=float32)}
{'predictions': array([11.042439], dtype=float32)}
{'predictions': array([11.099754], dtype=float32)}
{'predictions': array([11.09521], dtype=float32)}
{'predictions': array([11.041583], dtype=float32)}
{'predictions': array([11.042867], dtype=float32)}
{'predictions': array([11.042216], dtype=float32)}
{'predictions': array([11.098368], dtype=float32)}
{'predictions': array([11.040328], dtype=float32)}
{'predictions': array([11.043011], dtype=float32)}
{'predictions': array([11.323972], dtype=float32)}
{'predictions': array([11.041803], dtype=float32)}
{'predictions': array([11.040709], dtype=float32)}
{'predictions': array([11.04062

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [10]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100));
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbf306e6ef0>, '_master': '', '_global_id_in_cluster': 0, '_num_ps_replicas': 0, '_task_type': 'worker', '_model_dir': 'taxi_trained', '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_tf_random_seed': None, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_task_id': 0, '_train_distribute': None, '_session_config': None, '_keep_checkpoint_max': 5}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 29842.77, ste

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [11]:
import datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  CONCAT(STRING(pickup_datetime), STRING(pickup_longitude), STRING(pickup_latitude), STRING(dropoff_latitude), STRING(dropoff_longitude)) AS key,
  DAYOFWEEK(pickup_datetime)*1.0 AS dayofweek,
  HOUR(pickup_datetime)*1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND ABS(HASH(pickup_datetime)) % 4 == {1}".format(base_query, phase)
  else:
    query = "{0} AND ABS(HASH(pickup_datetime)) % {1} == {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).to_dataframe()

In [12]:
print_rmse(model, df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-21-03:42:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-6071
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-21-03:42:53
INFO:tensorflow:Saving dict for global step 6071: average_loss = 88.57236, global_step = 6071, loss = 11257.773
RMSE on dataset = 9.41128921508789


RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License